In [ ]:
## Import packages (yeah idk )
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import pearsonr
from scipy import stats
from sklearn import metrics
from sklearn.linear_model import LinearRegression # need
from sklearn.model_selection import train_test_split # need
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE
import sklearn.preprocessing as preprocessing
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler # need

### Random Forest Regression 



In [ ]:

from sklearn.ensemble import  RandomForestRegressor
# import alleged data set 

df = pd.read_csv("huh.csv")

#seperating our target and predictor 
X_rfr = df['predictor1', 'predictor2', 'predictor3']
y_rfr = df['target']

# make train test splits
X_rfr_train, X_rfr_test, y_rfr_train, y_rfr_test = train_test_split(X_rfr, y_rfr, test_size=0.1, random_state = 42)

# making things scaler
scaler = StandardScaler()
scaler.fit(X_rfr_train)

X_rfr_train_sc = scaler.transform(X_rfr_train)
X_rfr_test_sc = scaler.transform(X_rfr_test)

# create an object of the RandomForestRegressor
model_RFR = RandomForestRegressor(n_estimators = 100)

# fit the model with the training data
model_RFR.fit(X_rfr_train_sc, y_rfr_train)

# predict the target on train and test data
predict_train = model_RFR.predict(X_rfr_train_sc)
predict_test = model_RFR.predict(X_rfr_test_sc)

# Root Mean Squared Error on train and test data
print('RMSE on train data: ', metrics.mean_squared_error(y_rfr_train, predict_train)**(0.5))
print('RMSE on test data: ',  metrics.mean_squared_error(y_rfr_test, predict_test)**(0.5))

r2 = metrics.r2_score(y_rfr_test, predict_test)
print(r2)

# plot to show feature importances
plt.figure(figsize=(10,7))
feat_importances = pd.Series(model_RFR.feature_importances_, index = X_rfr_train.columns)
feat_importances.nlargest(7).plot(kind='barh');

### Linear Regression 

What if the relationship is linear ?

In [ ]:
from sklearn.linear_model import LinearRegression 

# import dataset 
df = pd.read_csv("huh.csv")

# seperating our target and predictor
X = df['predictor1', 'predictor2', 'predictor3'].to_numpy()
y = df['target'].to_numpy()

# seperating test train set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

lm = LinearRegression()
lm.fit(X_train_sc, y_train)
y_pred = lm.predict(X_test_sc)


print('RMSE on test data: ',  metrics.mean_squared_error(y_test, y_pred)**(0.5))
print('R^2:', metrics.r2_score(y_test, y_pred))

### Neural Networks

for funzies !!

In [ ]:
import sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA

#define RMSE as a function, since we'll use this in the NN model 
def rmse(target,prediction):
    return(np.sqrt(((target - prediction)**2).sum()/len(target)))


#import data
df = pd.read_csv('huh.csv')

# normalize data
df_norm = (data - data.mean())/data.std()
df_norm.tail()

#unpack variables
target = df_norm['target']
p2 = df_norm['p2']
p3 = df_norm['p3']
p4 = df_norm['p4']
p5 = df_norm['p5']

#target variable: y; predictor variable(s): x
y = target
y-=np.min(target)
y/=np.max(target) #now y ranges from 0 to 1
x = data_norm.drop('target',axis=1)

#first do PCA, then use PCs as predictors
n_modes = np.min(np.shape(x))
pca = PCA(n_components = n_modes)
PCs = pca.fit_transform(x)
eigvecs = pca.components_
fracVar = pca.explained_variance_ratio_

#plot fraction of variance explained by each mode

plt.figure(figsize=(5,5))

plt.subplot(1,1,1)
plt.scatter(range(len(fracVar)),fracVar)
plt.xlabel('Mode Number')
plt.ylabel('Fraction Variance Explained')
plt.title('Variance Explained by All Modes')

plt.tight_layout()

plt.show()

#take first 5 PCs in MLP NN model
#training: first 100 points
#testing: remaining points

ntrain = 100

x_train = PCs[:ntrain,:5] #train on 100 observations of first 5 PCs
y_train = y[:ntrain]

x_test = PCs[ntrain:,:5] #test on remaining observations
y_test = y[ntrain:]



In [ ]:
## OH GOD

#This chunk of code is relatively flexible and can be used to loop through a range of parameters
#We'll use this code several different times in the notebook
#Play around and see what kinds of models you can make

#First: MLP using 1 hidden layer with 10 neurons
#Second: MLP using 1 hidden layer with 1-10 neurons
#Third: something else!

##### Play around with these parameters
num_models = 10 #number of models to build for the ensemble
min_nhn = 1 #minimum number of hidden neurons to loop through (nhn = 'number hidden neurons')
max_nhn = 9 #maximum number of hidden neurons to loop through
max_hidden_layers = 1 #maximum number of hidden layers to loop through (nhl = 'number hidden layers')
batch_size = 32
solver = 'adam' #use stochastic gradient descent as an optimization method (weight updating algorithm)
activation = 'relu'
learning_rate_init = 0.01
#####

max_iter = 1500 #max number of epochs to run
early_stopping = True #True = stop early if validation error begins to rise
validation_fraction = 0.1 #fraction of training data to use as validation

y_out_all_nhn = []
y_out_ensemble = []
RMSE_ensemble = [] #RMSE for each model in the ensemble
RMSE_ensemble_cumsum = [] #RMSE of the cumulative saltation for each model
nhn_best = []
nhl_best = []

for model_num in range(num_models): #for each model in the ensemble
    
    print('Model Number: ' + str(model_num))
    
    RMSE = []
    y_out_all_nhn = []
    nhn = []
    nhl = []
    
    for num_hidden_layers in range(1,max_hidden_layers+1):
    
        print('\t # Hidden Layers = ' + str(num_hidden_layers))
    
        for num_hidden_neurons in range(min_nhn,max_nhn+1): #for each number of hidden neurons

            print('\t\t # hidden neurons = ' + str(num_hidden_neurons))
            
            hidden_layer_sizes = (num_hidden_neurons,num_hidden_layers)
            model = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, 
                                 verbose=False,
                                 max_iter=max_iter, 
                                 early_stopping = early_stopping,
                                 validation_fraction = validation_fraction,
                                 batch_size = batch_size,
                                 solver = solver,
                                 activation = activation,
                                 learning_rate_init = learning_rate_init)

            model.fit(x_train,y_train) #train the model

            y_out_this_nhn = model.predict(x_test) #model prediction for this number of hidden neurons (nhn)
            y_out_all_nhn.append(y_out_this_nhn) #store all models -- will select best one best on RMSE

            RMSE.append(rmse(y_test,y_out_this_nhn)) #RMSE between cumulative curves
            
            nhn.append(num_hidden_neurons)
            nhl.append(num_hidden_layers)
        
    indBest = RMSE.index(np.min(RMSE)) #index of model with lowest RMSE
    RMSE_ensemble.append(np.min(RMSE))
    nhn_best.append(nhn[indBest])
    nhl_best.append(nhl[indBest])
    #nhn_best.append(indBest+1) #the number of hidden neurons that achieved best model performance of this model iteration
    y_out_ensemble.append(y_out_all_nhn[indBest])
    
    print('\t BEST: ' + str(nhl_best[model_num]) + ' hidden layers, '+ str(nhn_best[model_num]) + ' hidden neurons')
    
y_out_ensemble_mean = np.mean(y_out_ensemble,axis=0)
RMSE_ensemble_mean = rmse(y_out_ensemble_mean,y_test)


In [ ]:

#visualize

plt.figure(figsize=(12,8))

plt.subplot(241)
plt.scatter(len(RMSE_ensemble),RMSE_ensemble_mean,c='k',marker='*')
plt.scatter(range(len(RMSE_ensemble)),RMSE_ensemble)
plt.xlabel('Model #')
plt.ylabel('RMSE')
plt.title('Error')

plt.subplot(242)
plt.scatter(range(len(nhn_best)),nhn_best)
plt.xlabel('Model #')
plt.ylabel('# Hidden Neurons')
plt.title('Hidden Neurons')

plt.subplot(243)
plt.scatter(range(len(nhl_best)),nhl_best)
plt.xlabel('Model #')
plt.ylabel('# Hidden Layers')
plt.title('Hidden Layers')

plt.subplot(244)
plt.scatter(y_test,y_out_ensemble_mean)
#plt.plot((np.min(y_test),np.max(y_test)),'k--')
plt.xlabel('y_test')
plt.ylabel('y_model')
plt.title('Ensemble')

plt.subplot(212)
plt.plot(y_out_ensemble_mean)
plt.plot(np.array(y_test),alpha = 0.5)

plt.tight_layout()

#visualize individual model runs


plt.figure(figsize = (12,5))

plt.scatter(range(len(y_test)),y_test,label='Observations',zorder = 0,alpha = 0.3)
plt.plot(range(len(y_test)),np.transpose(y_out_ensemble[0]),
         color = 'k',alpha = 0.4,label='Individual Models',zorder=1) #plot first ensemble member with a label
plt.plot(range(len(y_test)),np.transpose(y_out_ensemble[1:]),
         color = 'k',alpha = 0.4,zorder=1) #plot remaining ensemble members without labels for a nicer legend
plt.plot(range(len(y_test)),y_out_ensemble_mean,color = 'k',label = 'Ensemble',zorder=2, linewidth = 3)
plt.xlabel('Time', fontsize = 20)
plt.ylabel('y', fontsize = 20)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.title('MLP Model Results', fontsize = 24)
plt.legend(fontsize = 16, loc = 'best')

plt.tight_layout()

plt.show()

#visualize performance metrics/etc


plt.figure(figsize=(16,4))

plt.subplot(131)
plt.scatter(len(RMSE_ensemble),RMSE_ensemble_mean,c='k',marker='*', s = 150)
plt.scatter(range(len(RMSE_ensemble)),RMSE_ensemble, s = 150)
plt.xlabel('Model #', fontsize = 20)
plt.ylabel('RMSE', fontsize = 20)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
#plt.ylim((np.min(RMSE_ensemble) - 0.005, np.max(RMSE_ensemble)+0.005))
plt.title('Error', fontsize = 20)

plt.subplot(132)
plt.scatter(range(len(nhn_best)),nhn_best, s = 150)
plt.xlabel('Model #', fontsize = 20)
plt.ylabel('# Hidden Neurons', fontsize = 20)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.title('Hidden Neurons', fontsize = 20)

plt.subplot(133)
plt.scatter(range(len(nhl_best)),nhl_best, s = 150)
plt.xlabel('Model #', fontsize = 20)
plt.ylabel('# Hidden Layers', fontsize = 20)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.title('Hidden Layers', fontsize = 20)

plt.tight_layout()

plt.show()